# Acquire Exercises

### *1. Using the code from the lesson as a guide and the REST API from https://python.zgulde.net/api/v1/items .*

In [107]:
# General imports for working on the notebook on time series

import pandas as pd
import requests
import os

In [3]:
# Taking a look at the json file produced from the url

response = requests.get('https://python.zgulde.net/api/v1/items')
response.json()

{'payload': {'items': [{'item_brand': 'Riceland',
    'item_id': 1,
    'item_name': 'Riceland American Jazmine Rice',
    'item_price': 0.84,
    'item_upc12': '35200264013',
    'item_upc14': '35200264013'},
   {'item_brand': 'Caress',
    'item_id': 2,
    'item_name': 'Caress Velvet Bliss Ultra Silkening Beauty Bar - 6 Ct',
    'item_price': 6.44,
    'item_upc12': '11111065925',
    'item_upc14': '11111065925'},
   {'item_brand': 'Earths Best',
    'item_id': 3,
    'item_name': 'Earths Best Organic Fruit Yogurt Smoothie Mixed Berry',
    'item_price': 2.43,
    'item_upc12': '23923330139',
    'item_upc14': '23923330139'},
   {'item_brand': 'Boars Head',
    'item_id': 4,
    'item_name': 'Boars Head Sliced White American Cheese - 120 Ct',
    'item_price': 3.14,
    'item_upc12': '208528800007',
    'item_upc14': '208528800007'},
   {'item_brand': 'Back To Nature',
    'item_id': 5,
    'item_name': 'Back To Nature Gluten Free White Cheddar Rice Thin Crackers',
    'item_price':

### *In the following cells I am trying to understand and remember the content of the current course by going through the code myself*

In [4]:
# Doodling around checking the content of 'payload'

response.json()['payload']

{'items': [{'item_brand': 'Riceland',
   'item_id': 1,
   'item_name': 'Riceland American Jazmine Rice',
   'item_price': 0.84,
   'item_upc12': '35200264013',
   'item_upc14': '35200264013'},
  {'item_brand': 'Caress',
   'item_id': 2,
   'item_name': 'Caress Velvet Bliss Ultra Silkening Beauty Bar - 6 Ct',
   'item_price': 6.44,
   'item_upc12': '11111065925',
   'item_upc14': '11111065925'},
  {'item_brand': 'Earths Best',
   'item_id': 3,
   'item_name': 'Earths Best Organic Fruit Yogurt Smoothie Mixed Berry',
   'item_price': 2.43,
   'item_upc12': '23923330139',
   'item_upc14': '23923330139'},
  {'item_brand': 'Boars Head',
   'item_id': 4,
   'item_name': 'Boars Head Sliced White American Cheese - 120 Ct',
   'item_price': 3.14,
   'item_upc12': '208528800007',
   'item_upc14': '208528800007'},
  {'item_brand': 'Back To Nature',
   'item_id': 5,
   'item_name': 'Back To Nature Gluten Free White Cheddar Rice Thin Crackers',
   'item_price': 2.61,
   'item_upc12': '759283100036',

In [5]:
# Assigning 'response.json()' to a variable and checking out the keys

data = response.json()
data.keys()

dict_keys(['payload', 'status'])

In [7]:
# Checking the keys out

data['status'].keys()

'ok'

In [12]:
# Checking the keys out of payload

data['payload'].keys()

dict_keys(['items', 'max_page', 'next_page', 'page', 'previous_page'])

In [15]:
# Displaying items in the data

data['payload']['items']

# Data['payload']['items']['num':'num'] EQUIVALENT because dict

[{'item_brand': 'Riceland',
  'item_id': 1,
  'item_name': 'Riceland American Jazmine Rice',
  'item_price': 0.84,
  'item_upc12': '35200264013',
  'item_upc14': '35200264013'},
 {'item_brand': 'Caress',
  'item_id': 2,
  'item_name': 'Caress Velvet Bliss Ultra Silkening Beauty Bar - 6 Ct',
  'item_price': 6.44,
  'item_upc12': '11111065925',
  'item_upc14': '11111065925'},
 {'item_brand': 'Earths Best',
  'item_id': 3,
  'item_name': 'Earths Best Organic Fruit Yogurt Smoothie Mixed Berry',
  'item_price': 2.43,
  'item_upc12': '23923330139',
  'item_upc14': '23923330139'},
 {'item_brand': 'Boars Head',
  'item_id': 4,
  'item_name': 'Boars Head Sliced White American Cheese - 120 Ct',
  'item_price': 3.14,
  'item_upc12': '208528800007',
  'item_upc14': '208528800007'},
 {'item_brand': 'Back To Nature',
  'item_id': 5,
  'item_name': 'Back To Nature Gluten Free White Cheddar Rice Thin Crackers',
  'item_price': 2.61,
  'item_upc12': '759283100036',
  'item_upc14': '759283100036'},
 {'i

In [18]:
# Displaying the data as a df

pd.DataFrame(data['payload']['items'])

,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925
2,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixe...,2.43,23923330139,23923330139
3,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice ...,2.61,759283100036,759283100036
5,Sally Hansen,6,Sally Hansen Nail Color Magnetic 903 Silver El...,6.93,74170388732,74170388732
6,Twinings Of London,7,Twinings Of London Classics Lady Grey Tea - 20 Ct,9.64,70177154004,70177154004
7,Lea & Perrins,8,Lea & Perrins Marinade In-a-bag Cracked Pepper...,1.68,51600080015,51600080015
8,Van De Kamps,9,Van De Kamps Fillets Beer Battered - 10 Ct,1.79,19600923015,19600923015
9,Ahold,10,Ahold Cocoa Almonds,3.17,688267141676,688267141676


In [ ]:
# Decomposing the code to understand it and reproduce it in a function with a loop

# setup
# 1. Classroom example
domain = 'https://api.data.codeup.com'
endpoint = '/api/v1/items'
items = []

# 2. Exercise
domain = 'https://python.zgulde.net/api/v1/items'
endpoint = '/api/v1/items' # Would n beot needed if /api/v1/items
                            # is already connected to domain like on the
                            # previous line but for current setup is needed
items = []

# For each page -- until next page is None
url = domain + endpoint
response = requests.get(url)
data = response.json()
items.extend(data['payload']['items'])
# update the endpoint
endpoint = data['payload']['next_page']

In [32]:
# Getting data from each page until the last page (None)
def get_items():
    '''
    Using a for loop to get each page from
    the data via url until last page.
    Stops the loop when request gets a None
    at the end of the pages
    '''
    # Setting up variables
    domain = 'https://python.zgulde.net'
    endpoint = '/api/v1/items'
    items = []
        
    for i in endpoint:
        url = domain + endpoint
        response = requests.get(url)
        data = response.json()
        items.extend(data['payload']['items'])
        endpoint = data['payload']['next_page']
        if endpoint == None:
            break
    return pd.DataFrame(items)

In [33]:
# Printing what the function returns

get_items()

,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925
2,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixe...,2.43,23923330139,23923330139
3,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice ...,2.61,759283100036,759283100036
5,Sally Hansen,6,Sally Hansen Nail Color Magnetic 903 Silver El...,6.93,74170388732,74170388732
6,Twinings Of London,7,Twinings Of London Classics Lady Grey Tea - 20 Ct,9.64,70177154004,70177154004
7,Lea & Perrins,8,Lea & Perrins Marinade In-a-bag Cracked Pepper...,1.68,51600080015,51600080015
8,Van De Kamps,9,Van De Kamps Fillets Beer Battered - 10 Ct,1.79,19600923015,19600923015
9,Ahold,10,Ahold Cocoa Almonds,3.17,688267141676,688267141676


In [63]:
# Assigning 'get_items()' to a variable

items_df = get_items()

In [72]:
# Writing the new df to a local csv file

items_df.to_csv('items_df.csv')

## *2. Repeating the process for stores*

In [38]:
# Displaying payload

response = requests.get('https://api.data.codeup.com/api/v1/stores')
data = response.json()
data['payload']
# stores = pd.DataFrame(data['payload']['stores'])

{'max_page': 1,
 'next_page': None,
 'page': 1,
 'previous_page': None,
 'stores': [{'store_address': '12125 Alamo Ranch Pkwy',
   'store_city': 'San Antonio',
   'store_id': 1,
   'store_state': 'TX',
   'store_zipcode': '78253'},
  {'store_address': '9255 FM 471 West',
   'store_city': 'San Antonio',
   'store_id': 2,
   'store_state': 'TX',
   'store_zipcode': '78251'},
  {'store_address': '2118 Fredericksburg Rdj',
   'store_city': 'San Antonio',
   'store_id': 3,
   'store_state': 'TX',
   'store_zipcode': '78201'},
  {'store_address': '516 S Flores St',
   'store_city': 'San Antonio',
   'store_id': 4,
   'store_state': 'TX',
   'store_zipcode': '78204'},
  {'store_address': '1520 Austin Hwy',
   'store_city': 'San Antonio',
   'store_id': 5,
   'store_state': 'TX',
   'store_zipcode': '78218'},
  {'store_address': '1015 S WW White Rd',
   'store_city': 'San Antonio',
   'store_id': 6,
   'store_state': 'TX',
   'store_zipcode': '78220'},
  {'store_address': '12018 Perrin Beitel 

In [52]:
response = requests.get('https://api.data.codeup.com/api/v1/stores')
data = response.json()
data['payload']
stores1 = pd.DataFrame(data['payload']['stores'])

In [53]:
stores1

,store_address,store_city,store_id,store_state,store_zipcode
0,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
1,9255 FM 471 West,San Antonio,2,TX,78251
2,2118 Fredericksburg Rdj,San Antonio,3,TX,78201
3,516 S Flores St,San Antonio,4,TX,78204
4,1520 Austin Hwy,San Antonio,5,TX,78218
5,1015 S WW White Rd,San Antonio,6,TX,78220
6,12018 Perrin Beitel Rd,San Antonio,7,TX,78217
7,15000 San Pedro Ave,San Antonio,8,TX,78232
8,735 SW Military Dr,San Antonio,9,TX,78221
9,8503 NW Military Hwy,San Antonio,10,TX,78231


In [65]:
# Adapting the function for stores

def get_stores():
    '''
    Using a for loop to get each page from
    the data via url until last page.
    Stops the loop when request gets a None
    at the end of the pages
    '''
    # Setup
    domain = 'https://api.data.codeup.com'
    endpoint = '/api/v1/stores'
    stores = []
    for i in endpoint:
        url = domain + endpoint
        response = requests.get(url)
        data = response.json()
        stores.extend(data['payload']['stores'])
        endpoint = data['payload']['next_page']
        if endpoint == None:
            break
    return pd.DataFrame(stores)

In [66]:
get_stores()

,store_address,store_city,store_id,store_state,store_zipcode
0,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
1,9255 FM 471 West,San Antonio,2,TX,78251
2,2118 Fredericksburg Rdj,San Antonio,3,TX,78201
3,516 S Flores St,San Antonio,4,TX,78204
4,1520 Austin Hwy,San Antonio,5,TX,78218
5,1015 S WW White Rd,San Antonio,6,TX,78220
6,12018 Perrin Beitel Rd,San Antonio,7,TX,78217
7,15000 San Pedro Ave,San Antonio,8,TX,78232
8,735 SW Military Dr,San Antonio,9,TX,78221
9,8503 NW Military Hwy,San Antonio,10,TX,78231


In [67]:
stores_df = get_stores()

In [71]:
stores_df.to_csv('stores_df.csv')

## *3. Extract the data for sales (https://python.zgulde.net/api/v1/sales).*

**Repeating the same process as for the others**

In [57]:
# Adapting the function for sales

def get_sales():
    '''
    Using a for loop to get each page from
    the data via url until last page.
    Stops the loop when request gets a None
    at the end of the pages
    '''
    # Setup
    domain = 'https://python.zgulde.net'
    endpoint = '/api/v1/sales'
    sales = []
    for i in endpoint:
        url = domain + endpoint
        response = requests.get(url)
        data = response.json()
        sales.extend(data['payload']['sales'])
        # Update the endpoint
        endpoint = data['payload']['next_page']
        if endpoint == None:
            break
    return pd.DataFrame(sales)

# https://python.zgulde.net/api/v1/sales

In [70]:
sales_df = get_sales()
sales_df.head()

,item,sale_amount,sale_date,sale_id,store
0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,1
1,1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,1
2,1,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",3,1
3,1,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",4,1
4,1,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",5,1


In [123]:
sales_df.to_csv('sales_df.csv', index=False)

## *4. Save the data in your files to local csv files*

>### Done along with each function setup

## *5. Combine the data from your three separate dataframes into one large dataframe.*

>### *Displaying the different dataframes to take an ensemble view*

In [74]:
items_df.head()

,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925
2,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixe...,2.43,23923330139,23923330139
3,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice ...,2.61,759283100036,759283100036


In [76]:
sales_df.head()

,item,sale_amount,sale_date,sale_id,store
0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,1
1,1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,1
2,1,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",3,1
3,1,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",4,1
4,1,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",5,1


In [75]:
stores_df.head()

,store_address,store_city,store_id,store_state,store_zipcode
0,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
1,9255 FM 471 West,San Antonio,2,TX,78251
2,2118 Fredericksburg Rdj,San Antonio,3,TX,78201
3,516 S Flores St,San Antonio,4,TX,78204
4,1520 Austin Hwy,San Antonio,5,TX,78218


In [80]:
# Making sure another column but the first one isn't the index

sales_df.index

RangeIndex(start=0, stop=65000, step=1)

In [81]:
# Renaming the columns to avoid confusion and enable merger

sales_df.rename(columns={'item': 'item_id', 'store': 'store_id'}, inplace=True)

In [82]:
# Displaying new sales_df

sales_df.head()

,item_id,sale_amount,sale_date,sale_id,store_id
0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,1
1,1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,1
2,1,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",3,1
3,1,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",4,1
4,1,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",5,1


In [84]:
# Putting the dataframes together

df = pd.merge(sales_df, items_df, how='left', on='item_id')
sales_info = pd.merge(df, stores_df, how='left', on='store_id')

In [85]:
sales_info.head()

,item_id,sale_amount,sale_date,sale_id,store_id,item_brand,item_name,item_price,item_upc12,item_upc14,store_address,store_city,store_state,store_zipcode
0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253
1,1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253
2,1,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",3,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253
3,1,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",4,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253
4,1,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",5,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253


### *Checking the unique store ids and items ids*

In [86]:
sales_info.store_id.nunique()

10

In [87]:
sales_info.item_id.nunique()

4

In [88]:
# Getting general information from the df

sales_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 65000 entries, 0 to 64999
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   item_id        65000 non-null  int64  
 1   sale_amount    65000 non-null  float64
 2   sale_date      65000 non-null  object 
 3   sale_id        65000 non-null  int64  
 4   store_id       65000 non-null  int64  
 5   item_brand     65000 non-null  object 
 6   item_name      65000 non-null  object 
 7   item_price     65000 non-null  float64
 8   item_upc12     65000 non-null  object 
 9   item_upc14     65000 non-null  object 
 10  store_address  65000 non-null  object 
 11  store_city     65000 non-null  object 
 12  store_state    65000 non-null  object 
 13  store_zipcode  65000 non-null  object 
dtypes: float64(2), int64(3), object(9)
memory usage: 7.4+ MB


## *6. Acquire the Open Power Systems Data for Germany*

In [97]:
# Taking a look at the json file produced from the url

response = requests.get('https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv')
response.json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [105]:
# Getting the data from the url

opsd = pd.read_csv('https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv') 

In [106]:
opsd.head()

,Date,Consumption,Wind,Solar,Wind+Solar
0,2006-01-01,1069.184,NaN,NaN,NaN
1,2006-01-02,1380.521,NaN,NaN,NaN
2,2006-01-03,1442.533,NaN,NaN,NaN
3,2006-01-04,1457.217,NaN,NaN,NaN
4,2006-01-05,1477.131,NaN,NaN,NaN


In [108]:
# Writing the acquisition function for Open Power Systems Data for Germany

def get_opsd_germany():
    if os.path.exists('opsd.csv'):
        return pd.read_csv('opsd.csv')
    df = pd.read_csv('https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv')
    df.to_csv('opsd.csv', index=False)
    return df

In [109]:
opsd_df = get_opsd_germany()

In [110]:
opsd_df.head()

,Date,Consumption,Wind,Solar,Wind+Solar
0,2006-01-01,1069.184,NaN,NaN,NaN
1,2006-01-02,1380.521,NaN,NaN,NaN
2,2006-01-03,1442.533,NaN,NaN,NaN
3,2006-01-04,1457.217,NaN,NaN,NaN
4,2006-01-05,1477.131,NaN,NaN,NaN


In [112]:
opsd_df.rename(columns={'Wind+Solar' : 'Wind_and_Solar'}, inplace=True)

In [113]:
opsd_df.head()

,Date,Consumption,Wind,Solar,Wind_and_Solar
0,2006-01-01,1069.184,NaN,NaN,NaN
1,2006-01-02,1380.521,NaN,NaN,NaN
2,2006-01-03,1442.533,NaN,NaN,NaN
3,2006-01-04,1457.217,NaN,NaN,NaN
4,2006-01-05,1477.131,NaN,NaN,NaN


In [124]:
opsd_df.to_csv('opsd_df.csv', index=False)

### Attempts at more generalizing function

In [118]:
# Getting api data regardless of endpoint

def get_api_data(endpoint):
    '''
    Using a for loop to get each page from
    the data via url until last page.
    Stops the loop when request gets a None
    at the end of the pages
    '''
    # Setup
    domain = 'https://python.zgulde.net'
    extension = '/api/v1/'
    endpoint = 'sales'
    sales = []
    for i in endpoint:
        url = domain + extension + endpoint
        response = requests.get(url)
        data = response.json()
        sales.extend(data['payload']['sales'])
        # Update the endpoint
        endpoint = data['payload']['next_page']
        if endpoint == None:
            break
    return df

In [121]:
url

'https://python.zgulde.net/api/v1/items?page=3'

In [122]:
get_api_data(sales)

KeyError: 'payload'